# **Neural Network for Image Classification**
*   Implementation of a classic neural network for image classification supporting **multi-precision** training.
    *   Implementation currently supports training in either double, single, or half precision.
    *   This implies that both the computations and parameter storage are done in the specified precision.
*   Implementation of a classic neural network for image classification supporting **mixed-precision** training.
    *   Implementation currently supports half precision computations with single precision parameter storage.
*   Implementations are based off of TensorFlow's very own example: [TensorFlow Image Claasification](https://www.tensorflow.org/tutorials/keras/classification)

In [1]:
import time
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# Set a global TF random seed
tf.random.set_seed(12)

In [2]:
# Define number of training runs to compute the average training time over
NUM_TRAINING_RUNS = 5
# Values are specific to the Fashion MNIST dataset
INPUT_DIM = 28
NUM_CLASSES = 10

In [3]:
def build_and_train(X_train, y_train, precision):
    if precision == 'double':
        dtype = tf.float64
    elif precision == 'single':
        dtype = tf.float32
    else: # half
        dtype = tf.float16

    model = models.Sequential([
        layers.Flatten(input_shape=(INPUT_DIM, INPUT_DIM)),
        layers.Dense(128, activation='relu', dtype=dtype),
        layers.Dense(NUM_CLASSES, dtype=dtype)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])
    
    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    return model, training_time

In [4]:
def build_and_train_mixed(X_train, y_train):
    tf.keras.mixed_precision.set_global_policy('mixed_float16')
    model = models.Sequential([
        layers.Flatten(input_shape=(INPUT_DIM, INPUT_DIM)),
        layers.Dense(128, activation='relu'),
        layers.Dense(NUM_CLASSES)
    ])
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

    start_time = time.time()
    model.fit(X_train, y_train, epochs=5)
    end_time = time.time()
    training_time = end_time - start_time

    tf.keras.mixed_precision.set_global_policy('float32')
    return model, training_time

In [5]:
# Load dataset and split into train and test sets
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Scale values before feeding into neural net
X_train = X_train / 255.0
X_test  = X_test / 255.0

In [ ]:
# Test run to make sure that everything is working properly before starting actual measurements
_ = build_and_train(X_train, y_train, precision='single')
_ = build_and_train_mixed(X_train, y_train)

In [6]:
# Train with double precision
time_double = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_double, training_time = build_and_train(X_train, y_train, 'double')
    time_double += training_time
accuracy_double = model_double.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4985 - accuracy: 0.8257
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3679 - accuracy: 0.8683
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3337 - accuracy: 0.8787
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3120 - accuracy: 0.8855
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2935 - accuracy: 0.8909
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4995 - accuracy: 0.8242
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3722 - accuracy: 0.8661
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3352 - accuracy: 0.8777
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3127 - accuracy: 0.8857
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2948 - accurac

In [7]:
# Train with single precision
time_single = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_single, training_time = build_and_train(X_train, y_train, 'single')
    time_single += training_time
accuracy_single = model_single.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.5019 - accuracy: 0.8230
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3754 - accuracy: 0.8645
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3379 - accuracy: 0.8782
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3146 - accuracy: 0.8837
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2948 - accuracy: 0.8908
Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4996 - accuracy: 0.8235
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3733 - accuracy: 0.8647
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3345 - accuracy: 0.8784
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3136 - accuracy: 0.8854
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2949 - accurac

In [8]:
# Train with half precision
time_half = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_half, training_time = build_and_train(X_train, y_train, 'half')
    time_half += training_time
accuracy_half = model_half.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 6s 2ms/step - loss: 19.0189 - accuracy: 0.6873
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 3.4242 - accuracy: 0.7836
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 2.6212 - accuracy: 0.7988
Epoch 4/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.1600 - accuracy: 0.8119
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.9285 - accuracy: 0.8164
Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 17.7464 - accuracy: 0.6802
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 3.6308 - accuracy: 0.7764
Epoch 3/5
1875/1875 [==============================] - 5s 2ms/step - loss: 2.4356 - accuracy: 0.7973
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 1.9762 - accuracy: 0.8085
Epoch 5/5
1875/1875 [==============================] - 5s 2ms/step - loss: 1.6820 - accur

In [9]:
# Train with mixed half precision
time_mixed = 0.0
for _ in range(NUM_TRAINING_RUNS):
    model_mixed, training_time = build_and_train_mixed(X_train, y_train)
    time_mixed += training_time
accuracy_mixed = model_mixed.evaluate(X_test, y_test, verbose=2)[1]

Epoch 1/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.5001 - accuracy: 0.8242
Epoch 2/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3740 - accuracy: 0.8640
Epoch 3/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3387 - accuracy: 0.8760
Epoch 4/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3143 - accuracy: 0.8853
Epoch 5/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2941 - accuracy: 0.8918
Epoch 1/5
1875/1875 [==============================] - 7s 3ms/step - loss: 0.4932 - accuracy: 0.8255
Epoch 2/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3743 - accuracy: 0.8645
Epoch 3/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.3359 - accuracy: 0.8767
Epoch 4/5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.3124 - accuracy: 0.8854
Epoch 5/5
1875/1875 [==============================] - 6s 3ms/step - loss: 0.2935 - accurac

In [10]:
print("---RESULTS---")
print("Average training time in double precision:", time_double / NUM_TRAINING_RUNS, "seconds")
print("Average training time in single precision:", time_single/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in half precision:", time_half/ NUM_TRAINING_RUNS, "seconds")
print("Average training time in mixed half precision:", time_mixed/ NUM_TRAINING_RUNS, "seconds")
print("-------------")
print("Accuracy with double precision:", accuracy_double)
print("Accuracy with single precision:", accuracy_single)
print("Accuracy with half precision:", accuracy_half)
print("Accuracy with mixed half precision:", accuracy_mixed)

---RESULTS---
Average training time in double precision: 41.96263284683228 seconds
Average training time in single precision: 38.48553071022034 seconds
Average training time in half precision: 35.364750146865845 seconds
Average training time in mixed half precision: 37.08986630439758 seconds
-------------
Accuracy with double precision: 0.8707000017166138
Accuracy with single precision: 0.8738999962806702
Accuracy with half precision: 0.8738999962806702
Accuracy with mixed half precision: 0.8776000142097473
